In [1]:
#loading location data files
import csv
with open('iot_locations.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file, delimiter=';')
    next(csv_reader)
    iot_locations = {row[0]: row[1] for row in csv_reader}
with open('simulation_locations.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file, delimiter=';')
    next(csv_reader)
    simulation_locations = {
        row[0]: {"location": row[1], "property": row[2]}
        for row in csv_reader
    }
with open('iot_and_simulation_locations_mapping.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file, delimiter=';')
    next(csv_reader)  # Skip the header line
    iot_and_simulation_locations_mapping = {}
    for row in csv_reader:
        key = row[0] if row[0] else f"no_iot_zone{len(iot_and_simulation_locations_mapping)}"
        iot_and_simulation_locations_mapping[key] = row[1]


In [2]:
#loading ellonasoft api iot data
import requests
from datetime import datetime, timedelta
import credentials 
iot_data=[]
url = 'https://ellonasoft.io/api/v2/login'
token=''
payload = {
    "login":credentials.ellonasoft_login,
    "password":credentials.ellonasoft_password
}
headers = {
    'Content-Type': 'application/json'
}
end_date = (datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
start_date = (datetime.now() - timedelta(minutes=120)).strftime("%Y-%m-%dT%H:%M:%S")
response = requests.post(url, json=payload, headers=headers)
if response.status_code == 200:
    data = response.json()
    token=response.json()['token']
    url = 'https://ellonasoft.io/api/v2/extract-data'
    payload = {
        "from": start_date,
        "to": end_date,
    }
    headers = {
        'Content-Type': 'application/json',
        'x-auth': token
    }
    response = requests.post(url, json=payload, headers=headers)
    devices=response.json()['data']['devices']
    measurements=response.json()['data']['measurements'][-1]
    for device in devices:
        if device+'-env_temp' in measurements:
            location = iot_locations.get(devices[device], '')
            iot_data.append({
                'device_id': devices[device],
                'location':location,
                'property': 'temperature',
                'value' : measurements[device + '-env_temp'],
                'unit' : response.json()['data']['units']['temperature'],
                'timestamp':measurements['date']
            })
            iot_data.append({
                'device_id': devices[device],
                'location':location,
                'property': 'humidity',
                'value' : measurements[device + '-env_rh'],
                'unit' : response.json()['data']['units']['humidity'],
                'timestamp':measurements['date']
            })
            iot_data.append({
                'device_id': devices[device],
                'location':location,
                'property': 'pressure',
                'value' : measurements[device + '-env_pres'],
                'unit' : response.json()['data']['units']['barometer'],
                'timestamp':measurements['date']
            })
            iot_data.append({
                'device_id': devices[device],
                'location':location,
                'property': 'carbon_dioxide',
                'value' : measurements[device + '-co2_co2'],
                'unit' : response.json()['data']['units']['co2'],
                'timestamp':measurements['date']
            })
else:
    print(f"Failed to create resource: {response.status_code}")
    print("Response:", response.text)

In [3]:
#loading ethera api iot data

url = "https://s5.nemocloud.com/AirQualityAPI/session/login"
payload = (
    '{"company": "' + credentials.ethera_company + '", '
    '"operator": "' + credentials.ethera_operator + '", '
    '"password": "' + credentials.ethera_password + '", '
    '"uuid": "string", '
    '"version": "string"}'
)
headers = {
  'accept': 'application/json',
  'Authorization': f'Digest username=Test,realm=Authorized users of etheraApi,nonce={credentials.ethera_nonce},uri=/AirQualityAPI/session/login,response={credentials.ethera_response},opaque={credentials.ethera_opaque}',
  'Accept-version': 'v4',
  'Content-Type': 'application/json',
}
response = requests.request("POST", url, headers=headers, data=payload)
if response.status_code == 201:
    session_id=response.json()["sessionId"]
    url = "https://s5.nemocloud.com/AirQualityAPI/devices/"
    payload = {}
    headers = {
      'accept': 'application/json',
      'Accept-version': 'v4',
      'sessionId': session_id,
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        devices=response.json()
        for device in devices:
            url = f"https://s5.nemocloud.com/AirQualityAPI/devices/{device['serial']}/lastValues"
            payload = {}
            headers = {
              'accept': 'application/json',
              'Accept-version': 'v4',
              'sessionId': session_id,
            }
            response = requests.request("GET", url, headers=headers, data=payload)
            if response.status_code == 200:
                measurements=response.json();
                for measurement in measurements:
                    if measurement['variable']['name'] in ['Temperature','Pressure','Humidity','Carbon_dioxide']:
                        location = iot_locations.get(device['serial'], '')
                        iot_data.append({
                            'device_id': device['serial'],
                            'location': location,
                            'value':measurement['values'][0]['value'],
                            'unit' : measurement['variable']['unit'],
                            'property': measurement['variable']['name'].lower(),
                            'timestamp':datetime.fromtimestamp(measurement['values'][0]['time'] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
                        })
            else:
                print('\nDevice:'+device['name'],'\t Failed \t',response.status_code.response.text())
    else:
        print(f'Failed to get devices list: {response.status_code}')
else:
    print(f'Failed: {response.status_code}')

In [4]:
#loading wattsense api iot data
import base64
import hashlib
import hmac
import time
from urllib import parse
url = 'https://api.wattsense.com'
class WattsenseAuth(requests.auth.AuthBase):
    def __init__(self, api_key, api_secret):
        self.api_key = api_key
        self.api_secret = api_secret
    def __call__(self, r):
        timestamp = int(time.time() * 1000)
        url = parse.urlparse(r.url)
        message = '\n'.join([str(e) for e in [r.method, url.path,
                            (url.query if url.query else None),
                            (r.body.decode('utf-8') if r.body else None),
                            timestamp] if e != None])
        hmac_hash = base64.b64encode(hmac.new(self.api_secret.encode(), message.encode(),
                                     hashlib.sha512).digest()).decode()
        r.headers['X-API-Auth'] = '{}:{}'.format(self.api_key, hmac_hash)
        r.headers['X-API-Timestamp'] = str(timestamp)
        return r
req = requests.get(f'{url}/v1/devices/K0XRLNlG/properties', auth=WattsenseAuth(api_key=credentials.wattsense_api_key, api_secret=credentials.wattsense_api_secret))
wattsenseData = []
i=0
while req.status_code == 200 and i<30:
    i+=1;
    for item in req.json():
        wattsenseData.append(item)
    if req.links.get('next'):
        next_url_query = parse.urlparse(req.links['next']['url']).query
        params = dict(x.split('=') for x in next_url_query.split('&'))
        params['since']= int((time.time()-600) * 1000);
        params['until']= int(time.time() * 1000);
        req = requests.get(f'{url}/v1/devices/K0XRLNlG/properties', params=params, auth=WattsenseAuth(api_key=credentials.wattsense_api_key, api_secret=credentials.wattsense_api_secret))
    else:
        break
for item in wattsenseData:
    if "temp_" in item['name']:
        slug=item['slug'].split('-')
        device_id = slug[0]+(('-'+slug[1]) if any(char.isdigit() for char in slug[1]) else "")
        location = iot_locations.get(device_id, '')
        value=item['payload']
        m_property = "setpoint_temperature" if "set_temp" in item['name'] else "temperature"
        iot_data.append({
            "device_id":device_id,
            "location":location,
            "value":value,
            "unit":item['unit'],
            "property": m_property,
            "timestamp":datetime.fromtimestamp(item['timestamp'] / 1000.0).strftime('%Y-%m-%dT%H:%M:%S')
        })

In [5]:
#transform iot data
units={'temperature':'DEG_C','setpoint_temperature':'DEG_C','humidity':"PERCENT_RH",'pressure':'HectoPA','carbon_dioxide':"PPM"}
transformed_iot_data = {"locations": {}}
for item in iot_data:
    location_name = item['location']
    device_id = item['device_id']
    if location_name not in transformed_iot_data["locations"]:
        transformed_iot_data["locations"][location_name] = {"location_name":location_name,"devices": {}}
    if device_id not in transformed_iot_data["locations"][location_name]["devices"]:
        transformed_iot_data["locations"][location_name]["devices"][device_id] = {
            "device_id": device_id,
            "properties": {}
        }
    transformed_iot_data["locations"][location_name]["devices"][device_id]["properties"][item['property']] = {
        "property":item['property'],
        "value": item['value'],
        "unit": units.get(item['property']),
        "timestamp": item['timestamp']
    }

In [12]:
print(simulation_data)

{'simulationModels': {'K15_0bws_0sim_0A_Studies_ScheduledSp_02_sp1': {'FeatureOfInterest': 'K15_0bws_0sim_0A_Studies_ScheduledSp_02_sp1', 'version': None, 'format': 'FMU', 'formatVersion': '2.0', 'author': None, 'copyright': None, 'licence': None, 'generationTool': 'Dymola Version 2023x Refresh 1, 2023-04-12 (requires license to execute)', 'generationDateAndTime': '2024-12-17T23:37:29Z', 'simulation': {0: {'simulationVariables': {'T_air_0001': {'type': 'output', 'location': 'roof', 'property': 'temperature'}, 'T_air_0002': {'type': 'output', 'location': 'zone001', 'property': 'temperature'}, 'T_air_0003': {'type': 'output', 'location': 'zone002', 'property': 'temperature'}, 'T_air_0004': {'type': 'output', 'location': 'zone005', 'property': 'temperature'}, 'T_air_0005': {'type': 'output', 'location': 'zone004', 'property': 'temperature'}, 'T_air_0006': {'type': 'output', 'location': 'zone003', 'property': 'temperature'}, 'T_air_0007': {'type': 'output', 'location': 'zone101', 'property

In [13]:
print(transformed_iot_data)

{'locations': {'office011': {'location_name': 'office011', 'devices': {'POD2-00335': {'device_id': 'POD2-00335', 'properties': {'temperature': {'property': 'temperature', 'value': 21.556521954743758, 'unit': 'DEG_C', 'timestamp': '2024-12-20T08:35:00.000Z'}, 'humidity': {'property': 'humidity', 'value': 36.72608649212381, 'unit': 'PERCENT_RH', 'timestamp': '2024-12-20T08:35:00.000Z'}, 'pressure': {'property': 'pressure', 'value': 1015.852175505265, 'unit': 'HectoPA', 'timestamp': '2024-12-20T08:35:00.000Z'}, 'carbon_dioxide': {'property': 'carbon_dioxide', 'value': 375.7391304347826, 'unit': 'PPM', 'timestamp': '2024-12-20T08:35:00.000Z'}}}, 'xdppivjzy2wixy6s-l1301': {'device_id': 'xdppivjzy2wixy6s-l1301', 'properties': {'setpoint_temperature': {'property': 'setpoint_temperature', 'value': 22.0, 'unit': 'DEG_C', 'timestamp': '2024-12-20T10:28:48'}, 'temperature': {'property': 'temperature', 'value': 20.700000762939453, 'unit': 'DEG_C', 'timestamp': '2024-12-20T10:28:48'}}}}}, 'office02

In [11]:
#loading simulation model data
import fmpy
from fmpy import read_model_description
from fmpy.model_description import ModelDescription

fmu_file_path = "EDF_building_thermal_model.fmu"
model_description: ModelDescription = read_model_description(fmu_file_path)
variables = model_description.modelVariables
simulation_data = {
    "simulationModels":{
        model_description.modelName:{
            'FeatureOfInterest':model_description.modelName,
            'FeatureOfInterest':model_description.modelName,
            'version':model_description.version,
            'format':'FMU',
            'formatVersion':model_description.fmiVersion,
            'author':model_description.author,
            'copyright':model_description.copyright,
            'licence':model_description.license,
            'generationTool':model_description.generationTool,
            'generationDateAndTime':model_description.generationDateAndTime,
            'simulation':{
                0:{
                'simulationVariables':{}
                }
            }
        }
    }
}
for variable in variables:
    location=simulation_locations.get(variable.name,{}).get('location',"")
    variable_property=simulation_locations.get(variable.name,{}).get('property',"")
    if location!="" or variable_property!="":
        causality = getattr(variable, "causality", None)
        variability = getattr(variable, "variability", None)
        if causality == "input":
            simulation_data['simulationModels'][model_description.modelName]['simulation'][0]['simulationVariables'][variable.name]={"type":"input","location":location,"property":variable_property}
        elif causality == "output":
            simulation_data['simulationModels'][model_description.modelName]['simulation'][0]['simulationVariables'][variable.name]={"type":"output","location":location,"property":variable_property}
        elif causality == "parameter" or variability == "parameter":
            simulation_data['simulationModels'][model_description.modelName]['simulation'][0]['simulationVariables'][variable.name]={"type":"parameter","location":location,"property":variable_property}


In [13]:
#Generate the knowledge graph
from rdflib import Graph, Namespace, Literal, URIRef,BNode
from rdflib.namespace import RDF,XSD
import owlrl

def uri_compatible(string):
    formatted_string = ""
    for char in string:
        if char.isalnum() or char in "-_":
            formatted_string += char
        else:
            formatted_string += "_" 
    return formatted_string

def unique_uri(graph, base_uri, suffix_format="_{:d}"):
    unique_uri = URIRef(base_uri)
    suffix = 1
    
    while (unique_uri, None, None) in graph or (None, None, unique_uri) in graph:
        unique_uri = URIRef(base_uri + suffix_format.format(suffix))
        suffix += 1
    return unique_uri



SAREF = Namespace("https://saref.etsi.org/core/")
WOSO = Namespace("https://purl.org/woso/")
EX = Namespace("http://example.org/iot/")
UNIT= Namespace("http://qudt.org/vocab/unit/")
g = Graph()
g.bind("saref", SAREF)
g.bind("woso", WOSO)
g.bind("unit", UNIT)
g.bind("ex", EX)

for iot_foi_id,simulation_foi_id in iot_and_simulation_locations_mapping.items():
    if "no_iot_zone" in iot_foi_id:
        simulation_foi_uri=URIRef(EX[simulation_foi_id])
        g.add((simulation_foi_uri, RDF.type, SAREF.FeatureOfInterest))
    else:
        iot_foi_id = URIRef(EX[iot_foi_id])
        g.add((iot_foi_id, RDF.type, SAREF.FeatureOfInterest))
        if simulation_foi_id and EX[simulation_foi_id]==unique_uri(g,EX[simulation_foi_id]):
            g.add((EX[simulation_foi_id], RDF.type, SAREF.FeatureOfInterest))
        g.add((EX[simulation_foi_id], SAREF.consistsOf, iot_foi_id))

for foi_id,location in transformed_iot_data['locations'].items():
    foi_uri = URIRef(EX[foi_id]) if location != '' else BNode()
    for device_id,device in location['devices'].items():
        device_uri = URIRef(EX[device_id])
        g.add((device_uri, RDF.type, SAREF.Sensor))
        for property_kind_id,property_of_interest in device['properties'].items():
            property_kind_id=uri_compatible(property_kind_id)
            if EX[property_kind_id] == unique_uri(g,EX[property_kind_id]):
                property_kind_uri=URIRef(EX[property_kind_id])
                g.add((property_kind_uri, RDF.type, SAREF.property))
            property_of_interest_uri=URIRef(f'{foi_uri}#{property_kind_id}')
            g.add((property_of_interest_uri, RDF.type, SAREF.PropertyOfInterest))
            g.add((property_of_interest_uri, SAREF.hasPropertyKind, EX[property_kind_id]))
            g.add((property_of_interest_uri, SAREF.isPropertyOfInterestOf, foi_uri))    
            observation_uri=unique_uri(g,EX[f"{property_kind_id}_observation"])
            g.add((observation_uri, RDF.type, SAREF.Observation))    
            g.add((observation_uri, SAREF.madeBy, device_uri))
            g.add((observation_uri, SAREF.hasTimestamp, Literal(property_of_interest['timestamp'], datatype=XSD.dateTime)))
            g.add((observation_uri, SAREF.observes, property_of_interest_uri))
            property_value_uri=unique_uri(g,EX[f"{device_id}{property_kind_id.capitalize()}Value"],"{:d}")
            g.add((observation_uri, SAREF.hasResult, property_value_uri))
            g.add((property_value_uri, RDF.type, SAREF.PropertyValue))
            g.add((property_value_uri, SAREF.isValueOfProperty, property_of_interest_uri))
            g.add((property_value_uri, SAREF.hasValue, Literal(property_of_interest['value'],datatype=XSD.decimal)))
            g.add((property_value_uri, SAREF.isMeasuredIn, UNIT[property_of_interest['unit']]))

for simulation_model_id,model in simulation_data['simulationModels'].items():
    simulation_model_uri = URIRef(EX[simulation_model_id])
    g.add((simulation_model_uri, RDF.type, WOSO.SimulationModel))
    g.add((simulation_model_uri, WOSO.models, EX[model['FeatureOfInterest']]))
    g.add((simulation_model_uri, WOSO.version, Literal(model['version'],datatype=XSD.string)))
    g.add((simulation_model_uri, WOSO.formatVersion, Literal(model['formatVersion'],datatype=XSD.string)))
    g.add((simulation_model_uri, WOSO[format], Literal(model['format'],datatype=XSD.string)))
    g.add((simulation_model_uri, WOSO.author, Literal(model['author'],datatype=XSD.string)))
    g.add((simulation_model_uri, WOSO.copyright, Literal(model['copyright'],datatype=XSD.string)))
    g.add((simulation_model_uri, WOSO.licence, Literal(model['licence'],datatype=XSD.string)))
    g.add((simulation_model_uri, WOSO.generationTool, Literal(model['generationTool'],datatype=XSD.string)))
    g.add((simulation_model_uri, WOSO.generationToolDateAndTime, Literal(model['generationDateAndTime'],datatype=XSD.dateTime)))
    for key,simulation in model['simulation'].items():
        simulation_uri=unique_uri(g,EX[f"{simulation_model_id}Simulation"])
        g.add((simulation_uri, RDF.type, WOSO.Simulation))
        g.add((simulation_uri, WOSO.isExecutionOf, simulation_model_uri))
        for simulation_variable_id,simulation_variable in simulation['simulationVariables'].items():
            simulation_variable_uri=unique_uri(g,EX[f"{simulation_variable_id}"])
            g.add((simulation_variable_uri, RDF.type, WOSO.SimulationVariable))
            if simulation_variable['type']=="input":
                g.add((simulation_uri, SAREF.hasInput, simulation_variable_uri))
            if simulation_variable['type']=="output":
                g.add((simulation_uri, SAREF.hasOutput, simulation_variable_uri))
            if simulation_variable['type']=="parameter":
                g.add((simulation_uri, SAREF.hasParameter, simulation_variable_uri))
            if simulation_variable['type'] in ["parameter","input"]:
                property_kind_id=uri_compatible(simulation_variable["property"])
                if EX[property_kind_id] == unique_uri(g,EX[property_kind_id]):
                    property_kind_uri=URIRef(EX[property_kind_id])
                    g.add((property_kind_uri, RDF.type, SAREF.property))
                g.add((simulation_variable_uri, WOSO.isRelatedToProperty, EX[f'{simulation_variable["location"]}#{property_kind_id}']))
                property_of_interest_uri=EX[f'{simulation_variable["location"]}#{property_kind_id}']
                g.add((EX[simulation_variable["location"]], SAREF.hasPropertyOfInterest, property_of_interest_uri))
                g.add((EX[model['FeatureOfInterest']], SAREF.consistsOf, EX[simulation_variable["location"]]))
                g.add((property_of_interest_uri, SAREF.hasPropertyKind, EX[property_kind_id]))
            
g.update("""
INSERT { ?variable woso:hasValue ?value .}
WHERE {
{SELECT ?variable (avg(?values) AS ?value)
    WHERE{
        ?observation saref:observes ?propertyOfInterestIot.
        ?propertyValue saref:isValueOfProperty ?propertyOfInterestIot.
        ?propertyValue saref:hasValue ?values.
        ?featureOfInterestSimulation saref:consistsOf ?featureOfInterestIot.
        ?propertyOfInterestIot saref:isPropertyOfInterestOf ?featureOfInterestIot.
        ?propertyOfInterestIot saref:hasPropertyKind ?propertyKind.
        ?featureOfInterestSimulation saref:hasPropertyOfInterest ?propertyOfInterestSimulation.
        ?variable woso:isRelatedToProperty ?propertyOfInterestSimulation.
        ?propertyOfInterestSimulation saref:hasPropertyKind ?propertyKind.
        }GROUP BY ?variable
    }
    }


""")

g.serialize("EDF_building_data_KG.ttl", format="turtle")

#saref_url = "https://saref.etsi.org/core/v3.2.1/saref.ttl"
#woso_url = "https://zhounas.github.io/woso/woso.ttl"
#g.parse(saref_url, format="ttl")
#g.parse(woso_url, format="ttl")

with open("EDF_building_data_KG.ttl", "r+") as file:
    content = file.read()
    file.seek(0)
    file.write("@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n" + content)
